In [34]:
img_path=r'F:'
file_name=r'Project%d'
img_name=r'Region 1_Merged'
img_save_path=r'F:\Project_All'
save_img_name=r'Region 1_Merged%.4d'
save_path=r'C:\Users\dingj\ZhaoLab\20220806_SIFTvertStitch'
stitch_layer_num=3

dim_elem_num,dim_len,voxel_len=get_img_info(save_path)
axis_range_array=np.zeros((stitch_layer_num,6))
for i in range(1,stitch_layer_num+1):
    axis_range_array[i-1,:]=np.load(save_path+r'\axis_range_stitch_%.4d.npy'%(i)).reshape((1,-1))
xy_axis_range=np.zeros((2,2))
xy_axis_range[0,0],xy_axis_range[0,1]=np.min(axis_range_array[:,0])-voxel_len[0]*2,np.max(axis_range_array[:,1])+voxel_len[0]*100
xy_axis_range[1,0],xy_axis_range[1,1]=np.min(axis_range_array[:,2])-voxel_len[0]*2,np.max(axis_range_array[:,3])+voxel_len[0]*100
xy_voxel_num=np.int64(np.round((xy_axis_range[:,1]-xy_axis_range[:,0])/voxel_len[0:2])+1)
print(xy_voxel_num)

img_num=0
for i in range(stitch_layer_num):
    first_last_index=np.load(save_path+r'\first_last_index_stitch_%.4d.npy'%(i+1))
    axis_range=np.load(save_path+r'\axis_range_stitch_%.4d.npy'%(i+1))
    this_img_path=img_path+'\\'+file_name%(i+1)
    x_th=np.int64(np.round((axis_range[0,0]-xy_axis_range[0,0])/voxel_len[0]))
    y_th=np.int64(np.round((axis_range[1,0]-xy_axis_range[1,0])/voxel_len[1]))
    for j in range(first_last_index[0],first_last_index[1]):
        this_img=np.zeros((xy_voxel_num[1::-1]),dtype='uint8')
        img_2D=import_2D_img(this_img_path,img_name,j)
        if i==0:
            m1=np.mean(img_2D)
        if i==1:
            m2=np.mean(img_2D)
            img_2D=img_2D.astype('float32')
            img_2D=np.uint8(np.clip(m1/m2*img_2D,0,255))
        img_2D=np.uint8(0.8*img_2D.astype('float32'))
        this_img[y_th:y_th+img_2D.shape[0],x_th:x_th+img_2D.shape[1]]=img_2D
        cv2.imwrite(r'%s\Region 1_Merged%.4d.tif'%(img_save_path,img_num),this_img)
        img_num+=1

[3353 3825]


In [33]:
voxel_len

array([2.88793758e-06, 2.88804799e-06, 4.91644000e-06])

In [27]:
help(np.clip)

Help on function clip in module numpy:

clip(a, a_min, a_max, out=None, **kwargs)
    Clip (limit) the values in an array.
    
    Given an interval, values outside the interval are clipped to
    the interval edges.  For example, if an interval of ``[0, 1]``
    is specified, values smaller than 0 become 0, and values larger
    than 1 become 1.
    
    Equivalent to but faster than ``np.minimum(a_max, np.maximum(a, a_min))``.
    
    No check is performed to ensure ``a_min < a_max``.
    
    Parameters
    ----------
    a : array_like
        Array containing elements to clip.
    a_min, a_max : array_like or None
        Minimum and maximum value. If ``None``, clipping is not performed on
        the corresponding edge. Only one of `a_min` and `a_max` may be
        ``None``. Both are broadcast against `a`.
    out : ndarray, optional
        The results will be placed in this array. It may be the input
        array for in-place clipping.  `out` must be of the right shape
    

In [8]:
import numpy as np
import time
import cv2
import random

def get_img_info(save_path):
    dim_elem_num=np.load(save_path+r'\dim_elem_num.npy')
    dim_len=np.load(save_path+r'\dim_len.npy')
    voxel_len=np.load(save_path+r'\voxel_len.npy')
    return dim_elem_num,dim_len,voxel_len

def import_2D_img(img_path,img_name,z_th):
    one_img_name=r'%s\%s_z%.2d_RAW_ch00.tif'%(img_path,img_name,z_th)
    return cv2.imread(one_img_name,cv2.IMREAD_GRAYSCALE)